In [ ]:
import tensorflow as tf 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import pandas as pd 


data = pd.read_csv('letter-recognition.xls')

# Convert labels (letters) to numbers, ensuring only single character labels are processed
data['letter'] = data['letter'].apply(lambda x : ord(x[0]) - ord('A') if len(x) == 1 else -1 )

# Remove rows where the label is invalid (not a single character)
data = data[data['letter'] != -1]

#split the features and labels 
X = data.iloc[: , 1:].values
y = data.iloc[: , 0].values

# one hot encode
y = to_categorical(y , num_classes = 26)

# train Test Split 
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

# Normalize the features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

# Build the deep Neural network model
model = Sequential([
    Dense(128 , activation = 'relu' , input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64 , activation = 'relu'),
    Dropout(0.3),
    Dense(26 , activation = 'softmax')
])

#Compile the model
model.compile(optimizer='adam' ,
             loss = 'categorical_crossentropy' ,
             metrics=['accuracy'])

# train the model
history = model.fit(X_train , y_train , 
                   epochs = 30 ,
                   batch_size = 32 ,
                   validation_split = 0.2 ,
                   verbose = 1)

test_loss , test_accuracy = model.evaluate(X_test , y_test , verbose = 0)
print(f"Test Accuracy : {test_accuracy * 100:.2f}%")

predictions = model.predict(X_test)
y_pred = predictions.argmax(axis=1)
y_true = y_test.argmax(axis=1)

print("Classification Report : ")
print(classification_report(y_true , y_pred , target_names=[chr(i) for i in range (ord('A') , ord('Z') + 1)]))

from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=[chr(i) for i in range(65,91)], yticklabels=[chr(i) for i in range(65,91)])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualize training history
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()